In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# 让画图内置
%matplotlib inline

# 忽略版本号之类的warnings
import warnings
warnings.filterwarnings('ignore')
import os
from datetime import date

# 数据导入

In [3]:
# 导入数据
# original_data = pd.read_csv(r'data\2019-Oct.csv')
# data = original_data.copy()
# data.shape

In [3]:
# 导入数据
data = pd.read_csv(r'data\2019-Oct.csv')
data.shape

(42448764, 9)

In [16]:
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [4]:
data['category_code'] = data['category_code'].astype('category')
data['brand'] = data['brand'].astype('category')
# data['event_type'] = data['event_type'].astype('category')


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42448764 entries, 0 to 42448763
Data columns (total 9 columns):
 #   Column         Dtype   
---  ------         -----   
 0   event_time     object  
 1   event_type     category
 2   product_id     int64   
 3   category_id    int64   
 4   category_code  category
 5   brand          category
 6   price          float64 
 7   user_id        int64   
 8   user_session   object  
dtypes: category(3), float64(1), int64(3), object(2)
memory usage: 2.1+ GB


# 空值

In [10]:
data.isnull().sum()

event_time              0
event_type              0
product_id              0
category_id             0
category_code    13515609
brand             6117080
price                   0
user_id                 0
user_session            2
dtype: int64

# 重复值

**重复记录数**

In [5]:
# 重复的记录数 
data.duplicated().sum()  

30220

In [20]:
# 有重复值的记录
data[data.duplicated(keep=False)]  

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
3829,2019-10-01 02:25:40 UTC,view,1307116,2053013558920217191,computers.notebook,hp,463.07,548049635,83085f41-22c3-4f27-bc29-bfaaf460d902
3830,2019-10-01 02:25:40 UTC,view,1307116,2053013558920217191,computers.notebook,hp,463.07,548049635,83085f41-22c3-4f27-bc29-bfaaf460d902
3831,2019-10-01 02:25:40 UTC,view,1307116,2053013558920217191,computers.notebook,hp,463.07,548049635,83085f41-22c3-4f27-bc29-bfaaf460d902
3832,2019-10-01 02:25:40 UTC,view,1307116,2053013558920217191,computers.notebook,hp,463.07,548049635,83085f41-22c3-4f27-bc29-bfaaf460d902
8564,2019-10-01 02:34:48 UTC,view,17200828,2053013559792632471,furniture.living_room.sofa,NaN,926.41,555464369,8bf38319-436a-4637-8c84-6de4213f8a13
...,...,...,...,...,...,...,...,...,...
42433267,2019-10-31 22:03:56 UTC,cart,1005115,2053013555631882655,electronics.smartphone,apple,949.47,515327682,259bbbbd-a170-4ef5-b134-4ae224375eeb
42434549,2019-10-31 22:11:49 UTC,view,5700662,2053013553970938175,auto.accessories.player,NaN,180.18,566267881,49436ad5-fda6-439a-9b4f-5b323888be97
42434551,2019-10-31 22:11:49 UTC,view,5700662,2053013553970938175,auto.accessories.player,NaN,180.18,566267881,49436ad5-fda6-439a-9b4f-5b323888be97
42443756,2019-10-31 23:17:58 UTC,cart,1004249,2053013555631882655,electronics.smartphone,apple,722.40,528350072,592de524-5929-4fa3-94be-dbae6c59a0f2


**不考虑 user_session 时的重复记录**

In [6]:
# 不考虑 session 时，重复记录数
data_no_session = data.iloc[:,:8]
data_no_session.duplicated().sum() # 4987

35207

In [5]:
# 除了session 不同，其他都相同的记录
data[data_no_session.duplicated(keep=False)]

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
3829,2019-10-01 02:25:40 UTC,view,1307116,2053013558920217191,computers.notebook,hp,463.07,548049635,83085f41-22c3-4f27-bc29-bfaaf460d902
3830,2019-10-01 02:25:40 UTC,view,1307116,2053013558920217191,computers.notebook,hp,463.07,548049635,83085f41-22c3-4f27-bc29-bfaaf460d902
3831,2019-10-01 02:25:40 UTC,view,1307116,2053013558920217191,computers.notebook,hp,463.07,548049635,83085f41-22c3-4f27-bc29-bfaaf460d902
3832,2019-10-01 02:25:40 UTC,view,1307116,2053013558920217191,computers.notebook,hp,463.07,548049635,83085f41-22c3-4f27-bc29-bfaaf460d902
6677,2019-10-01 02:31:22 UTC,view,1004777,2053013555631882655,electronics.smartphone,xiaomi,136.40,554907878,8eca030b-fe9a-4772-b189-e1e5443e4550
...,...,...,...,...,...,...,...,...,...
42436383,2019-10-31 22:23:06 UTC,view,4803977,2053013554658804075,electronics.audio.headphone,samsung,107.45,510783995,8a57abda-524c-44ee-b68c-1d4036dbd708
42442133,2019-10-31 23:05:05 UTC,view,10301191,2053013553115300101,NaN,chicco,7.72,563743967,7aa4435d-82ca-4ced-aaea-8548ffa8d2ac
42442134,2019-10-31 23:05:05 UTC,view,10301191,2053013553115300101,NaN,chicco,7.72,563743967,6af4130b-d38f-43af-b9cb-8a7cf224b5b4
42443756,2019-10-31 23:17:58 UTC,cart,1004249,2053013555631882655,electronics.smartphone,apple,722.40,528350072,592de524-5929-4fa3-94be-dbae6c59a0f2


**去除重复值**

In [6]:
# 去除重复值
""" 
keep='first' 给第二次出现的记录 true
我要去掉第二次及以后出现的记录，所以直接取反即可
"""

# 所有字段重复才删除
# data.drop_duplicates(keep='first', inplace=True)


# 前8个字段重复就删除
data =data[~data_no_session.duplicated(keep='first')]
data.shape  # 42413557 


(42413557, 9)

# 异常值

## 修改时间格式

In [9]:
data['event_time'] = pd.to_datetime(data['event_time']).dt.tz_localize(None)

## 去掉 user_session 列

In [14]:
data.drop(columns=['user_session'], inplace=True)

## 拆分商品类别字段

In [18]:
def extract_categorycode(input_text,level=0):
    '''
     this function splits category code and returns the first part.
    '''
    output_text=input_text.split('.')[level]
    return output_text

In [19]:
data['category'] = data['category_code'].apply(extract_categorycode,level=0)
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,category
0,2019-10-01 00:00:00,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,NaN
1,2019-10-01 00:00:00,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,appliances
2,2019-10-01 00:00:01,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,furniture
3,2019-10-01 00:00:01,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,computers
4,2019-10-01 00:00:04,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,electronics


In [20]:
data['sub_category'] = data['category_code'].apply(extract_categorycode,level=1)
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,category,sub_category
0,2019-10-01 00:00:00,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,NaN,NaN
1,2019-10-01 00:00:00,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,appliances,environment
2,2019-10-01 00:00:01,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,furniture,living_room
3,2019-10-01 00:00:01,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,computers,notebook
4,2019-10-01 00:00:04,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,electronics,smartphone


In [21]:
data['product_name'] = data['category_code'].apply(extract_categorycode,level=-1)
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,category,sub_category,product_name
0,2019-10-01 00:00:00,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,NaN,NaN,NaN
1,2019-10-01 00:00:00,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,appliances,environment,water_heater
2,2019-10-01 00:00:01,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,furniture,living_room,sofa
3,2019-10-01 00:00:01,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,computers,notebook,notebook
4,2019-10-01 00:00:04,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,electronics,smartphone,smartphone


# 保存数据

In [22]:
data

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,category,sub_category,product_name
0,2019-10-01 00:00:00,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,NaN,NaN,NaN
1,2019-10-01 00:00:00,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,appliances,environment,water_heater
2,2019-10-01 00:00:01,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,furniture,living_room,sofa
3,2019-10-01 00:00:01,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,computers,notebook,notebook
4,2019-10-01 00:00:04,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,electronics,smartphone,smartphone
...,...,...,...,...,...,...,...,...,...,...,...
42448759,2019-10-31 23:59:58,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,electronics,camera,video
42448760,2019-10-31 23:59:58,view,10800172,2053013554994348409,NaN,redmond,61.75,527322328,NaN,NaN,NaN
42448761,2019-10-31 23:59:58,view,5701038,2053013553970938175,auto.accessories.player,kenwood,128.70,566280422,auto,accessories,player
42448762,2019-10-31 23:59:59,view,21407424,2053013561579406073,electronics.clocks,tissot,689.85,513118352,electronics,clocks,clocks


In [23]:
data.to_csv('data\processed_data.csv',index=False)